In [1]:
# Date    : 20/02/2023
# Author  : Sivuyile Sifuba
# Email   : sivuyilesifuba@gmail.com

# References
# [1] “Lenet-5 | Lenet-5 Architecture | Introduction to Lenet-5,” Analytics Vidhya,
# Mar. 18, 2021. https://www.analyticsvidhya.com/blog/2021/03/the-architecture-of-lenet-5/
#
# [2] He, Kaiming & Zhang, Xiangyu & Ren, Shaoqing & Sun, Jian. (2015). 
# Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet 
# Classification. IEEE International Conference on Computer Vision (ICCV 2015). 
# 1502. 10.1109/ICCV.2015.123.

In [5]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import ZeroPadding2D, Conv2D, PReLU, AveragePooling2D, Flatten, Dense 
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import set_random_seed, to_categorical

In [3]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])

In [7]:
# This ensures that the model is deterministic
set_random_seed(seed=6)

# The model implementations is as described in [1]. The Model is a 5 weight layer 
# network also known as LeNet-5. We have chosen to implement the network with a PReLU
# activation function as introduced by Kaiming He et al [2]. The PReLu function
# is said to have the capability of reducing overfitting and assists in model
# convergence [2]. The network will be used to classify MNIST digits. In contrast 
# to the original network in [1] we have opted for using the ADAM optimizer instead
# of SGD.

model_input = Input(shape=(28, 28, 1))
# (28, 28, 1)
x = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(model_input)
# (32, 32, 1)
x = Conv2D(filters=6, kernel_size=(5,5), strides=(1, 1))(x)
# (28, 28, 6)
x = PReLU()(x)
x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(x)
# (14, 14, 6)
x = Conv2D(filters=16, kernel_size=(5,5), strides=(1, 1))(x)
# (10, 10, 16)
x = PReLU()(x)
x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(x)
# (5, 5, 16)
x = Conv2D(filters=120, kernel_size=(5,5), strides=(1, 1), activation='tanh')(x)
# (1, 1, 120)
x = PReLU()(x)
x = Flatten()(x)
# (120)
x = Dense(units=84, activation='tanh')(x)
# (84)
model_output  = Dense(units=10, activation='softmax')(x)
# (10)

# Create a Keras Model
LeNet = Model(inputs=model_input , outputs=model_output)
LeNet.summary()
LeNet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model with input images and labels
LeNet.fit(x=x_train,
          y=y_train,
          validation_data=(x_test, y_test),
          epochs=10,
          batch_size=32)

# model accuracy on test dataset
score = LeNet.evaluate(x_test,
                       y_test,
                       batch_size=32,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 32, 32, 1)        0         
 ng2D)                                                           
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 p_re_lu_3 (PReLU)           (None, 28, 28, 6)         4704      
                                                                 
 average_pooling2d_2 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_4 (Conv2D)           (None, 10, 10, 16)        2416